In [54]:
# Import chunk
import pandas as pd
import scanpy as sc
import numpy as np
import os 
import warnings

In [55]:
# Settings chunk
warnings.filterwarnings("ignore")

In [123]:
def integrate_h5(h5_directory:str, verbose:bool, cluster_res:float, n_pcs:int):
    '''
    Automatically integrates h5 files in designated directory and reads it into the current workspace as scanpy adata
    '''

    # Access h5 directory information
    samples = [] 
    os.chdir(h5_directory)
    if verbose: print(f'Changed working directory to {os.getcwd()}')
    n_samples = len(os.listdir())
    if verbose: print(f'Found {n_samples} sample(s). Appending paths to sample list.')
    for file in os.listdir():
        samples.append(os.path.realpath(file))

    # Preprocess h5 samples 
    adata_dict = {}
    for sample in samples:
        name = os.path.basename(sample)
        if verbose: print(f'Preprocessing sample: {name}')
        adata = pp(sample,n_pcs=n_pcs,cluster_res=cluster_res)
        adata_dict[name] = adata
        adata_dict[name].obs['batch'] = name

    # Find intersections between samples we start with the sample with the most clusters
    if len(adata_dict.keys()) == 1:
        adata = adata_dict[0]
        return adata
        if verbose: print("Single dataset processed.")
    else: #### THIS is where i'm automating the sliding intersection -> ingestion steps. complicated nested loop? -DM ####
        if verbose: print("Converging datasets.")
        cluster_numbers = dict()
        for adata in adata_dict.values():
            name = adata.obs['batch'].unique()[0]
            clusters = len(adata.obs['leiden'].unique())
            cluster_numbers[name] = clusters
        cluster_numbers = sorted(cluster_numbers.items(), key = lambda item: item[1], reverse = True)[:2]
        init = 0
        push = 2
        top = cluster_numbers[init,push]
        if verbose: print(f'Converging sample {top[1]} into {top[0]}')

        
        print(f'Samples with the most clusters: {top2}')


        





def pp(h5_path,n_pcs,cluster_res):
    '''
    Standard preprocessing, from https://github.com/mousepixels/sanbomics_scripts/blob/main/simple_scanpy_integration.ipynb
    '''
    adata = sc.read_10x_h5(h5_path)
    adata.var_names_make_unique()
    sc.pp.filter_cells(adata, min_genes=200) #get rid of cells with fewer than 200 genes
    sc.pp.filter_genes(adata, min_cells=3) #get rid of genes that are found in fewer than 3 cells
    adata.var['mt'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
    upper_lim = np.quantile(adata.obs.n_genes_by_counts.values, .98)
    lower_lim = np.quantile(adata.obs.n_genes_by_counts.values, .02)
    adata = adata[(adata.obs.n_genes_by_counts < upper_lim) & (adata.obs.n_genes_by_counts > lower_lim)]
    adata = adata[adata.obs.pct_counts_mt < 20]
    sc.pp.normalize_total(adata, target_sum=1e4) #normalize every cell to 10,000 UMI
    sc.pp.log1p(adata) #change to log counts
    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5) #these are default values
    adata.raw = adata #save raw data before processing values and further filtering
    adata = adata[:, adata.var.highly_variable] #filter highly variable
    sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt']) #Regress out effects of total counts per cell and the percentage of mitochondrial genes expressed
    sc.pp.scale(adata, max_value=10) #scale each gene to unit variance
    sc.tl.pca(adata, svd_solver='arpack')
    sc.pp.neighbors(adata, n_neighbors=10, n_pcs= n_pcs)
    sc.tl.leiden(adata, resolution = cluster_res)
    sc.tl.umap(adata)
    return adata

    

In [124]:
integrate_h5('/home/dm2763/projects/scenic/pyscenic/OE_final/Filtered_mtx/MOLNG2407', cluster_res=0.5, n_pcs=50, verbose=True)

Changed working directory to /home/dm2763/projects/scenic/pyscenic/OE_final/Filtered_mtx/MOLNG2407
Found 4 sample(s). Appending paths to sample list.
Preprocessing sample: L34687
Preprocessing sample: L34688
Preprocessing sample: L34689
Preprocessing sample: L34690
Converging datasets.
Samples with the most clusters: [('L34687', 18), ('L34688', 17)]
